# Chatbot with Memory using LangChain + SQLite

## Objective
พัฒนาแชทบอทที่ขับเคลื่อนด้วย **LLM** พร้อมระบบความจำแบบ **Persistent** โดยใช้:
- **LangChain** - Framework สำหรับ LLM applications
- **SQLite** - เก็บประวัติการสนทนาแบบถาวร
- **Gemini API** - Google's LLM

## Features
- ✅ Persistent Memory - ปิด notebook แล้วเปิดใหม่ยังจำได้
- ✅ Sliding Window - เก็บบทสนทนา 3 รอบล่าสุด
- ✅ Session Management - แยก session ได้
- ✅ Production-ready architecture

## Architecture
```
User Input → LangChain → Gemini API → Response
                ↓              ↑
         SQLite Memory ←──────┘
         (Persistent)
```

## 1. Install Dependencies

In [4]:
# Install required packages
%pip install langchain langchain-google-genai langchain-community python-dotenv -q


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries

In [5]:
import os
import sqlite3
from datetime import datetime
from typing import List, Dict

from dotenv import load_dotenv

# LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

# Load environment variables
load_dotenv()

print("Libraries imported successfully!")

Libraries imported successfully!


## 3. Configuration

In [6]:
# Configuration
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SQLITE_DB_PATH = "chat_memory.db"
MAX_MEMORY_TURNS = 3  # Remember last 3 conversation turns

if not GEMINI_API_KEY:
    raise ValueError("Please set GEMINI_API_KEY in .env file")

print(f"✅ API Key loaded")
print(f"✅ SQLite DB: {SQLITE_DB_PATH}")
print(f"✅ Memory turns: {MAX_MEMORY_TURNS}")

✅ API Key loaded
✅ SQLite DB: chat_memory.db
✅ Memory turns: 3


## 4. SQLite Chat History with Sliding Window

สร้าง custom class ที่ extend SQLChatMessageHistory เพื่อจำกัดจำนวน messages

In [7]:
class SlidingWindowSQLChatHistory(SQLChatMessageHistory):
    """
    SQLite-based chat history with sliding window.
    Keeps only the last N conversation turns.
    """
    
    def __init__(self, session_id: str, db_path: str, max_turns: int = 3):
        """
        Initialize with sliding window support.
        
        Args:
            session_id: Unique session identifier
            db_path: Path to SQLite database
            max_turns: Maximum conversation turns to keep (1 turn = 2 messages)
        """
        super().__init__(
            session_id=session_id,
            connection_string=f"sqlite:///{db_path}"
        )
        self.max_turns = max_turns
        self.max_messages = max_turns * 2
    
    def add_message(self, message) -> None:
        """Add message and trim if necessary."""
        super().add_message(message)
        self._trim_messages()
    
    def _trim_messages(self):
        """Keep only the last max_messages."""
        messages = self.messages
        if len(messages) > self.max_messages:
            # Clear and re-add only the last max_messages
            self.clear()
            for msg in messages[-self.max_messages:]:
                super().add_message(msg)
    
    def get_turns_count(self) -> int:
        """Get number of complete conversation turns."""
        return len(self.messages) // 2
    
    def display_history(self):
        """Display formatted conversation history."""
        print("\n" + "=" * 50)
        print("CONVERSATION HISTORY (SQLite)")
        print("=" * 50)
        
        if not self.messages:
            print("(No messages)")
        else:
            for msg in self.messages:
                if isinstance(msg, HumanMessage):
                    print(f"👤 USER: {msg.content}")
                elif isinstance(msg, AIMessage):
                    print(f"🤖 BOT: {msg.content}")
        
        print("=" * 50 + "\n")


print("SlidingWindowSQLChatHistory class defined!")

SlidingWindowSQLChatHistory class defined!


## 5. Initialize LangChain Components

In [8]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.7
)

# Create prompt template with message history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant that remembers our conversation.
Be friendly, concise, and use the conversation context to provide relevant responses.
If the user asks about something mentioned earlier, refer to that context."""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create chain
chain = prompt | llm

print("✅ LangChain components initialized!")

✅ LangChain components initialized!


## 6. Create Chatbot with Memory

In [9]:
# Session store for multiple sessions
session_store = {}

def get_session_history(session_id: str) -> SlidingWindowSQLChatHistory:
    """
    Get or create chat history for a session.
    Uses SQLite for persistent storage.
    """
    if session_id not in session_store:
        session_store[session_id] = SlidingWindowSQLChatHistory(
            session_id=session_id,
            db_path=SQLITE_DB_PATH,
            max_turns=MAX_MEMORY_TURNS
        )
    return session_store[session_id]


# Create runnable with message history
chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

print("✅ Chatbot with SQLite memory created!")

✅ Chatbot with SQLite memory created!


## 7. Helper Functions

In [10]:
def chat(message: str, session_id: str = "default") -> str:
    """
    Send a message to the chatbot.
    
    Args:
        message: User's message
        session_id: Session identifier (default: "default")
    
    Returns:
        Bot's response
    """
    config = {"configurable": {"session_id": session_id}}
    response = chatbot.invoke({"input": message}, config=config)
    return response.content


def get_memory_status(session_id: str = "default") -> str:
    """Get current memory status."""
    history = get_session_history(session_id)
    return f"Memory: {history.get_turns_count()}/{MAX_MEMORY_TURNS} turns ({len(history.messages)} messages)"


def show_history(session_id: str = "default"):
    """Display conversation history."""
    history = get_session_history(session_id)
    history.display_history()


def clear_history(session_id: str = "default"):
    """Clear conversation history."""
    history = get_session_history(session_id)
    history.clear()
    print(f"✅ History cleared for session: {session_id}")


print("Helper functions defined!")

Helper functions defined!


## 8. Demo: Test Memory Feature

In [11]:
# Clear previous session for clean demo
clear_history("demo")

print("=" * 60)
print("DEMO: Testing Chatbot with SQLite Memory")
print("=" * 60)

# Test conversation
demo_messages = [
    "สวัสดีครับ ผมชื่อสมชาย",
    "ผมชอบกินส้มตำมากครับ",
    "ช่วยแนะนำร้านอาหารหน่อยได้ไหม",
    "ผมชื่ออะไรนะ?",      # Test: Should remember name
    "ผมชอบกินอะไร?"       # Test: Should remember food preference
]

for i, msg in enumerate(demo_messages, 1):
    print(f"\n--- Turn {i} ---")
    print(f"👤 USER: {msg}")
    response = chat(msg, session_id="demo")
    print(f"🤖 BOT: {response}")
    print(f"   [{get_memory_status('demo')}]")

/var/folders/0h/48vbygj529b59twjsw2byllc0000gn/T/ipykernel_52480/114407993.py:10: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  session_store[session_id] = SlidingWindowSQLChatHistory(


✅ History cleared for session: demo
DEMO: Testing Chatbot with SQLite Memory

--- Turn 1 ---
👤 USER: สวัสดีครับ ผมชื่อสมชาย
🤖 BOT: สวัสดีครับคุณสมชาย ยินดีที่ได้รู้จักครับ มีอะไรให้ผมช่วยวันนี้ครับ?
   [Memory: 1/3 turns (2 messages)]

--- Turn 2 ---
👤 USER: ผมชอบกินส้มตำมากครับ
🤖 BOT: ส้มตำอร่อยมากเลยครับ! คุณชอบส้มตำรสชาติแบบไหนครับ เผ็ดมาก เผ็ดน้อย หรือว่าชอบแบบใส่ปลาร้าครับ?
   [Memory: 2/3 turns (4 messages)]

--- Turn 3 ---
👤 USER: ช่วยแนะนำร้านอาหารหน่อยได้ไหม
🤖 BOT: ได้เลยครับ! เพื่อให้ผมแนะนำร้านอาหารได้ตรงใจคุณที่สุด รบกวนบอกข้อมูลเพิ่มเติมหน่อยได้ไหมครับ:

*   **คุณอยากทานอาหารประเภทไหน:** นอกจากส้มตำแล้ว สนใจอาหารประเภทอื่นด้วยไหมครับ เช่น อาหารไทย อาหารอิตาเลียน หรืออาหารญี่ปุ่น
*   **งบประมาณ:** อยากได้ร้านอาหารราคาประมาณเท่าไหร่ครับ
*   **ทำเลที่ตั้ง:** คุณสะดวกไปทานแถวไหนครับ

เมื่อได้ข้อมูลเหล่านี้แล้ว ผมจะช่วยแนะนำร้านอาหารที่เหมาะกับคุณได้เลยครับ
   [Memory: 3/3 turns (6 messages)]

--- Turn 4 ---
👤 USER: ผมชื่ออะไรนะ?
🤖 BOT: คุณชื่อสมชายครับ ผมจำได้ว่าคุณบอกผมตอนที่

In [ ]:
# Show conversation history stored in SQLite
show_history("demo")

## 9. Verify Persistent Storage

ตรวจสอบว่าข้อมูลถูกเก็บใน SQLite จริง

In [12]:
# Check SQLite database
print("=" * 60)
print("SQLite Database Contents")
print("=" * 60)

conn = sqlite3.connect(SQLITE_DB_PATH)
cursor = conn.cursor()

# Get all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f"Tables: {[t[0] for t in tables]}")

# Get message count
try:
    cursor.execute("SELECT COUNT(*) FROM message_store;")
    count = cursor.fetchone()[0]
    print(f"Total messages in DB: {count}")
    
    # Show recent messages
    cursor.execute("SELECT session_id, message FROM message_store ORDER BY id DESC LIMIT 5;")
    rows = cursor.fetchall()
    print("\nRecent messages:")
    for session_id, message in rows:
        print(f"  [{session_id}] {message[:80]}..." if len(message) > 80 else f"  [{session_id}] {message}")
except sqlite3.OperationalError as e:
    print(f"Note: {e}")

conn.close()
print("\n" + "=" * 60)

SQLite Database Contents
Tables: ['message_store']
Total messages in DB: 10

Recent messages:
  [demo] {"type": "ai", "data": {"content": "\u0e04\u0e38\u0e13\u0e1a\u0e2d\u0e01\u0e27\u...
  [demo] {"type": "human", "data": {"content": "\u0e1c\u0e21\u0e0a\u0e2d\u0e1a\u0e01\u0e3...
  [demo] {"type": "ai", "data": {"content": "\u0e04\u0e38\u0e13\u0e0a\u0e37\u0e48\u0e2d\u...
  [demo] {"type": "human", "data": {"content": "\u0e1c\u0e21\u0e0a\u0e37\u0e48\u0e2d\u0e2...
  [demo] {"type": "ai", "data": {"content": "\u0e44\u0e14\u0e49\u0e40\u0e25\u0e22\u0e04\u...



## 10. Multiple Sessions Demo

แสดงการใช้งานหลาย sessions พร้อมกัน

In [13]:
print("=" * 60)
print("DEMO: Multiple Sessions")
print("=" * 60)

# Session 1: User A
clear_history("user_a")
print("\n[Session: user_a]")
print(f"👤 USER A: ผมชื่อวิชัย")
print(f"🤖 BOT: {chat('ผมชื่อวิชัย', session_id='user_a')}")

# Session 2: User B
clear_history("user_b")
print("\n[Session: user_b]")
print(f"👤 USER B: ฉันชื่อสมศรี")
print(f"🤖 BOT: {chat('ฉันชื่อสมศรี', session_id='user_b')}")

# Ask both sessions about their names
print("\n--- Testing Memory Isolation ---")
print("\n[Session: user_a]")
print(f"👤 USER A: ผมชื่ออะไร?")
print(f"🤖 BOT: {chat('ผมชื่ออะไร?', session_id='user_a')}")

print("\n[Session: user_b]")
print(f"👤 USER B: ฉันชื่ออะไร?")
print(f"🤖 BOT: {chat('ฉันชื่ออะไร?', session_id='user_b')}")

DEMO: Multiple Sessions
✅ History cleared for session: user_a

[Session: user_a]
👤 USER A: ผมชื่อวิชัย
🤖 BOT: สวัสดีครับคุณวิชัย ยินดีที่ได้รู้จักครับ มีอะไรให้ผมช่วยเหลือวันนี้ครับ?
✅ History cleared for session: user_b

[Session: user_b]
👤 USER B: ฉันชื่อสมศรี
🤖 BOT: สวัสดีค่ะ คุณสมศรี ยินดีที่ได้รู้จักนะคะ มีอะไรให้ฉันช่วยเหลือวันนี้ไหมคะ?

--- Testing Memory Isolation ---

[Session: user_a]
👤 USER A: ผมชื่ออะไร?
🤖 BOT: คุณชื่อวิชัยครับ ผมจำได้ครับ :)

[Session: user_b]
👤 USER B: ฉันชื่ออะไร?
🤖 BOT: คุณชื่อสมศรีค่ะ เราเพิ่งคุยกันเมื่อกี้นี้เองค่ะ


## 11. Interactive Chat

In [14]:
def run_interactive_chat(session_id: str = "interactive"):
    """
    Run an interactive chat session.
    
    Commands:
    - 'quit' or 'exit': End session
    - 'history': Show conversation history
    - 'clear': Clear memory
    - 'status': Show memory status
    """
    print("\n" + "=" * 60)
    print(f"🤖 INTERACTIVE CHAT (Session: {session_id})")
    print("=" * 60)
    print("Commands: 'quit', 'history', 'clear', 'status'")
    print("=" * 60 + "\n")
    
    while True:
        try:
            user_input = input("👤 You: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() in ['quit', 'exit']:
                print("\n👋 Goodbye!")
                break
            elif user_input.lower() == 'history':
                show_history(session_id)
                continue
            elif user_input.lower() == 'clear':
                clear_history(session_id)
                continue
            elif user_input.lower() == 'status':
                print(f"   [{get_memory_status(session_id)}]")
                continue
            
            response = chat(user_input, session_id=session_id)
            print(f"🤖 Bot: {response}")
            print(f"   [{get_memory_status(session_id)}]\n")
            
        except KeyboardInterrupt:
            print("\n\n👋 Session interrupted. Goodbye!")
            break


# Uncomment to run interactive session
# run_interactive_chat()

## 12. Summary

สรุปการทำงานของ Chatbot with LangChain + SQLite

In [15]:
print("=" * 60)
print("CHATBOT WITH LANGCHAIN + SQLITE - SUMMARY")
print("=" * 60)

print("""
📋 Features Implemented:

1. ✅ LangChain Integration
   - ChatGoogleGenerativeAI (Gemini 2.0 Flash)
   - ChatPromptTemplate with MessagesPlaceholder
   - RunnableWithMessageHistory

2. ✅ SQLite Persistent Memory
   - SQLChatMessageHistory for storage
   - Custom SlidingWindowSQLChatHistory class
   - Data survives notebook restarts

3. ✅ Sliding Window Memory
   - Keeps last 3 conversation turns
   - Automatic trimming when limit exceeded

4. ✅ Multi-Session Support
   - Different users can have separate histories
   - Session isolation

📊 Configuration:
   - LLM: Gemini 2.0 Flash
   - Memory: SQLite (chat_memory.db)
   - Max Turns: 3 (= 6 messages)

🔧 Key Components:
   - SlidingWindowSQLChatHistory: Custom memory class
   - chat(): Send message and get response
   - show_history(): View stored messages
   - clear_history(): Reset conversation

💡 Advantages over In-Memory:
   - Persistent: Survives restarts
   - Scalable: Can handle large histories
   - Production-ready: Real database storage
""")

print("=" * 60)

CHATBOT WITH LANGCHAIN + SQLITE - SUMMARY

📋 Features Implemented:

1. ✅ LangChain Integration
   - ChatGoogleGenerativeAI (Gemini 2.0 Flash)
   - ChatPromptTemplate with MessagesPlaceholder
   - RunnableWithMessageHistory

2. ✅ SQLite Persistent Memory
   - SQLChatMessageHistory for storage
   - Custom SlidingWindowSQLChatHistory class
   - Data survives notebook restarts

3. ✅ Sliding Window Memory
   - Keeps last 3 conversation turns
   - Automatic trimming when limit exceeded

4. ✅ Multi-Session Support
   - Different users can have separate histories
   - Session isolation

📊 Configuration:
   - LLM: Gemini 2.0 Flash
   - Memory: SQLite (chat_memory.db)
   - Max Turns: 3 (= 6 messages)

🔧 Key Components:
   - SlidingWindowSQLChatHistory: Custom memory class
   - chat(): Send message and get response
   - show_history(): View stored messages
   - clear_history(): Reset conversation

💡 Advantages over In-Memory:
   - Persistent: Survives restarts
   - Scalable: Can handle large hist